### 1. Bài toán nhà máy

 Một nhà máy sản xuất hai loại sản phẩm (I) và (II) từ hai loại nguyên liệu A và B. Biết rằng mỗi sản phẩm loại I cần 4 đơn vị nguyên liệu A và 2 đơn vị nguyên liệu B; mỗi sản phẩm loại (II) cần 2 đơn vị nguyên liệu A và 4 đơn vị nguyên liệu B. Khi bán một sản phẩm loại I lãi 8 đơn vị tiền, khi bán 1 sản phẩm loại (II) lãi 6 đơn vị tiền. Hãy lập kế hoạch sản xuất sao cho thu lãi nhiều nhất với số dự trữ nguyên liệu có hạn: 60 đơn vị nguyên liệu A và 48 đơn vị nguyên liệu B. 

In [5]:
from ortools.init.python import init
from ortools.linear_solver import pywraplp

def main():
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver("GLOP")

    # Khởi tạo 2 biến x, y và khoảng giá trị
    x = solver.NumVar(0, 20, "x")
    y = solver.NumVar(0, 20, "y")

    infinity = solver.infinity()

    # Khởi tạo ràng buộc : 4x + 2y <= 60
    constraint1 = solver.Constraint(-infinity, 60, "ct")
    constraint1.SetCoefficient(x, 4) # 4x
    constraint1.SetCoefficient(y, 2) # 2y

    # Khởi tạo ràng buộc : 2x + 4y <= 48
    constraint2 = solver.Constraint(-infinity, 48, "ct")
    constraint2.SetCoefficient(x, 2) # 2x
    constraint2.SetCoefficient(y, 4) # 4y

    # Khai báo hàm mục tiêu : f(x, y) = 8x + 6y -> max
    objective = solver.Objective()
    objective.SetCoefficient(x, 8)
    objective.SetCoefficient(y, 6)
    objective.SetMaximization()

    print(f"Solving with {solver.SolverVersion()}")
    result_status = solver.Solve()

    print(f"Status: {result_status}")
    if result_status != pywraplp.Solver.OPTIMAL:
        print("The problem does not have an optimal solution!")
        if result_status == pywraplp.Solver.FEASIBLE:
            print("A potentially suboptimal solution was found")
        else:
            print("The solver could not solve the problem.")
            return
            

    print("Solution:")
    print("Objective value =", objective.Value())
    print("x =", x.solution_value())
    print("y =", y.solution_value())

    print("Advanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")
    
main()

Solving with Glop solver v9.11.4210
Status: 0
Solution:
Objective value = 132.0
x = 11.999999999999996
y = 6.000000000000003
Advanced usage:
Problem solved in 0 milliseconds
Problem solved in 2 iterations


### 2. Bài toán kỹ sư

Có n kỹ sư và n công việc. Mỗi kỹ sư làm mỗi công việc sẽ được trả công lao động khác nhau. Bài toán yêu cầu thuê mỗi công nhân thực thi một công việc sao cho tổng tiền công phải trả là thấp nhất 

In [1]:
import numpy as np
cost_matrix = np.array([
    [11, 14, 6],
    [8, 10, 11],
    [9, 12, 7]
])
for i in range(cost_matrix.shape[0]):
    min_value = np.min(cost_matrix[i])
    for j in range(cost_matrix.shape[1]):
        cost_matrix[i][j] -= min_value

print("Bảng chi phí sau khi trừ giá trị nhỏ nhất của từng hàng:")
print(cost_matrix)

for j in range(cost_matrix.shape[1]):
    min_value = np.min(cost_matrix[:, j])
    for i in range(cost_matrix.shape[0]):
        cost_matrix[i][j] -= min_value

print("Bảng chi phí sau khi trừ giá trị nhỏ nhất của từng cột:")
print(cost_matrix)

assigned_rows = [-1] * cost_matrix.shape[0]  # Chứa các công việc được phân cho từng hàng
assigned_cols = [-1] * cost_matrix.shape[1]  # Chứa các kỹ sư đã được phân công

for i in range(cost_matrix.shape[0]):
    for j in range(cost_matrix.shape[1]):
        if cost_matrix[i][j] == 0 and assigned_rows[i] == -1 and assigned_cols[j] == -1:
            assigned_rows[i] = j
            assigned_cols[j] = i

print("Phân công tối ưu:")
for i in range(len(assigned_rows)):
    print(f"Kỹ sư {i + 1} được phân công cho dự án {assigned_rows[i] + 1}")

# Tính tổng chi phí dựa trên phân công tìm được
total_cost = 0
for i in range(len(assigned_rows)):
    total_cost += cost_matrix[i, assigned_rows[i]] + np.min(cost_matrix[i])

print(f"Tổng chi phí tối ưu: {total_cost}")


Bảng chi phí sau khi trừ giá trị nhỏ nhất của từng hàng:
[[5 8 0]
 [0 2 3]
 [2 5 0]]
Bảng chi phí sau khi trừ giá trị nhỏ nhất của từng cột:
[[5 6 0]
 [0 0 3]
 [2 3 0]]
Phân công tối ưu:
Kỹ sư 1 được phân công cho dự án 3
Kỹ sư 2 được phân công cho dự án 1
Kỹ sư 3 được phân công cho dự án 0
Tổng chi phí tối ưu: 0


### 3. Bài toán Knapsack

Có n đồ vật, mỗi đồ vật có giá trị vi, trọng lượng wi . Một người có khả năng mang được trọng lượng W, hãy lựa chọn các đồ vật sao cho tổng trọng lượng các đồ vật nhỏ hơn W và tổng giá trị các đồ vật lớn nhất có thể. 

In [6]:
from ortools.linear_solver import pywraplp

def solve_knapsack_problem(values, weights, capacity):
    num_items = len(values)

    #sử dụng CBC
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        return None

    # Tạo biến quyết định x[i]: 1 nếu đồ vật i được chọn, 0 nếu không
    x = []
    for i in range(num_items):
        x.append(solver.BoolVar(f'x[{i}]'))

    # Ràng buộc: Tổng trọng lượng không vượt quá sức chứa của ba lô
    solver.Add(solver.Sum([weights[i] * x[i] for i in range(num_items)]) <= capacity)

    # Hàm mục tiêu: Tối đa hóa tổng giá trị
    objective = solver.Objective()
    for i in range(num_items):
        objective.SetCoefficient(x[i], values[i])
    objective.SetMaximization()

    # Giải bài toán
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Các đồ vật được chọn:')
        total_value = 0
        total_weight = 0
        for i in range(num_items):
            if x[i].solution_value() == 1:
                print(f'Đồ vật {i + 1} được chọn (giá trị: {values[i]}, trọng lượng: {weights[i]})')
                total_value += values[i]
                total_weight += weights[i]
        print(f'Tổng giá trị: {total_value}')
        print(f'Tổng trọng lượng: {total_weight}')
    else:
        print('Không tìm được giải pháp tối ưu!')

# Dữ liệu đầu vào
values = [60, 100, 120]  # vi
weights = [10, 20, 30]   # wi
capacity = 50            # W

solve_knapsack_problem(values, weights, capacity)


Các đồ vật được chọn:
Đồ vật 2 được chọn (giá trị: 100, trọng lượng: 20)
Đồ vật 3 được chọn (giá trị: 120, trọng lượng: 30)
Tổng giá trị: 220
Tổng trọng lượng: 50


### 4. Bài toán Knapsack

Lập phương án vận chuyển xăng từ 4 kho xăng đến 5 trạm tiêu thụ với chi phí vận chuyển, lượng xăng dự trữ tại mỗi kho và nhu cầu tiêu thụ xăng tại mỗi trạm được cho như bảng dưới đây sao cho tổng chi phí vận chuyển là nhỏ nhất. 

In [5]:
from ortools.linear_solver import pywraplp
def solve_transportation_problem(costs, supplies, demands):
    # Khỏi tạo
    solver = pywraplp.Solver.CreateSolver('GLOP')
    
    # x[i][j]
    x = []
    num_warehouses = len(supplies)
    num_stations = len(demands)
    
    for i in range(num_warehouses):
        x.append([solver.NumVar(0, solver.infinity(), f'x_{i}_{j}') for j in range(num_stations)])
    
    for i in range(num_warehouses):
        solver.Add(sum(x[i][j] for j in range(num_stations)) <= supplies[i])
    
    for j in range(num_stations):
        solver.Add(sum(x[i][j] for i in range(num_warehouses)) == demands[j])
    
    objective = solver.Objective()
    for i in range(num_warehouses):
        for j in range(num_stations):
            objective.SetCoefficient(x[i][j], costs[i][j])
    objective.SetMinimization()
    
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        total_cost = 0
        for i in range(num_warehouses):
            for j in range(num_stations):
                print(f'x[{i}][{j}] = {x[i][j].solution_value()}')
                total_cost += x[i][j].solution_value() * costs[i][j]
        print(f'Total transportation cost = {total_cost}')
    else:
        print('No solution found.')
costs = [
    [30, 27, 26, 9, 5],
    [27, 31, 27, 26, 7],
    [30, 34, 27, 4, 7],
    [30, 3, 8, 9, 10]
]
supplies = [10, 15, 5, 10]  
demands = [5, 4, 6, 8, 7]   
solve_transportation_problem(costs, supplies, demands)


Solution:
x[0][0] = 0.0
x[0][1] = 0.0
x[0][2] = 0.0
x[0][3] = 3.0
x[0][4] = 7.0
x[1][0] = 5.0
x[1][1] = 0.0
x[1][2] = 0.0
x[1][3] = 0.0
x[1][4] = 0.0
x[2][0] = 0.0
x[2][1] = 0.0
x[2][2] = 0.0
x[2][3] = 5.0
x[2][4] = 0.0
x[3][0] = 0.0
x[3][1] = 4.0
x[3][2] = 6.0
x[3][3] = 0.0
x[3][4] = 0.0
Total transportation cost = 277.0
